In [1]:
import numpy as np
from qiskit import QuantumCircuit, execute
from qiskit.providers.aer import QasmSimulator
from qiskit.visualization import *
import qiskit

In [20]:
num_qubits = 32

In [12]:
def create_random_states():

    alice_state = np.random.randint(0, 2, size=num_qubits)
    eve_basis = np.random.randint(0, 2, size=num_qubits)
    bob_basis = np.random.randint(0, 2, size=num_qubits)
#     print(f"Alice's State:\t {np.array2string(alice_state, separator='')}")
#     print(f"Eve's Basis:\t {np.array2string(eve_basis, separator='')}")
#     print(f"Bob's Basis:\t {np.array2string(bob_basis, separator='')}")
    
    return alice_state, eve_basis, bob_basis

In [13]:
# function to define the circuit for an exchange of bit string
def make_b92_circ(enc_state, meas_basis):
    
    num_qubits = len(enc_state)

    b92 = QuantumCircuit(num_qubits)

    for index in range(len(enc_state)):
      if enc_state[index] == 1:
        b92.h(index)

    
    b92.barrier()  

    for index in range(len(meas_basis)):
      if meas_basis[index] == 1:
        b92.h(index)

    # Do not change below this line
    b92.measure_all()
    return b92

In [14]:
# function to define the circuit for an exchange of bit string
def make_eaves_circ(enc_basis, enc_state, meas_basis):
    
    num_qubits = len(enc_state)

    b92 = QuantumCircuit(num_qubits)
        
    
    for index in range(num_qubits):
        if (enc_basis[index] == 0 and enc_state[index] == 1):
            b92.x(index)
        elif (enc_basis[index] == 1 and enc_state[index] == 0):
            b92.h(index)
        elif (enc_basis[index] == 1 and enc_state[index] == 1):
            b92.x(index)
            b92.h(index)
            
    for index in range(len(meas_basis)):
        if meas_basis[index] == 1:
            b92.h(index)
    
    
    b92.measure_all()
    return b92

In [15]:
def b92_sifting(enc_state, meas_basis, meas_result):
    
    num_qubits = len(enc_state)
    
    sender_key = ''
    receiver_key = ''
    
    for i in range(len(meas_result)):
      if meas_result[i] == '1':
        sender_key+=str(enc_state[i])
        receiver_key+=str((meas_basis[i]+1)%2)
        
    return (sender_key, receiver_key)



In [21]:
key_list_bob = []
diff_bits_rate = []
same_key = 0
iterations = 100

for index in range (0,iterations):

    alice_state, eve_basis, bob_basis = create_random_states()

    #Alice sends the bits to Bob but Eve eavesdrops in between
    b92_circ = make_b92_circ(alice_state, eve_basis)

    #final result got by Eve after measurement
    eve_res = execute(
            b92_circ.reverse_bits(),
            backend=QasmSimulator(),
            shots=1
        ).result().get_counts().most_frequent()

    eve_state = np.array(list(eve_res), dtype=int)

    # Eve updates the state to send it to Bob
#     eve_state = ''.join([str(int(eve_res[i]) ^ int(eve_basis[i])) for i in range(len(eve_res))])

#     eve_state = np.array(list(eve_state), dtype=int)

    #Bob measures the circuit thinking that it was sent by Alice
    b92_circ2 = make_eaves_circ(eve_basis, eve_state, bob_basis)

    bob_res = execute(
            b92_circ2.reverse_bits(),
            backend=QasmSimulator(),
            shots=1
        ).result().get_counts().most_frequent()

    
#     print(f"Eve's result:\t {np.array2string(eve_state, separator='')}")
#     print (f"Bob's measured result:\t {bob_res}")

    # key sifting takes place to get the final key to be used
    alice_key, bob_key = b92_sifting(alice_state, bob_basis, bob_res)
    
#     print(f"Alice's final key: \t{alice_key}")
#     print(f"Bob's final key: \t{bob_key}")
    key_list_bob.append(bob_key)
    
    same_bit = 0
    
    for i in range(len(bob_key)):
        if bob_key[i] == alice_key[i]:
            same_bit += 1
    diff_bits_rate.append((len(bob_key)-same_bit)/len(bob_key))
    
    if alice_key == bob_key:
        same_key += 1
    
avg_key_length = sum( map( len, key_list_bob))/ len( key_list_bob)
avg_bit_error = sum( diff_bits_rate)*100/ len( diff_bits_rate)
print(f"Initial key length = {num_qubits}")
print(f"Final average key length after {iterations} iterations =  {avg_key_length}")
print(f"Keys are same {same_key} times out of {iterations}")
print(f"Bit error rate is {avg_bit_error}%")


Simulation failed and returned the following error message:
ERROR:  [Experiment 0] QasmSimulator: Insufficient memory for 32-qubit circuit using "statevector" method. You could try using the "matrix_product_state" or "extended_stabilizer" method instead.


QiskitError: 'ERROR:  [Experiment 0] QasmSimulator: Insufficient memory for 32-qubit circuit using "statevector" method. You could try using the "matrix_product_state" or "extended_stabilizer" method instead. ,  ERROR: QasmSimulator: Insufficient memory for 32-qubit circuit using "statevector" method. You could try using the "matrix_product_state" or "extended_stabilizer" method instead.'